In [1]:
import pandas as pd
import numpy as np

# خواندن فایل CSV
df = pd.read_csv("merged_output2.csv")

# تبدیل ستون DATE_MILADI به نوع datetime
df['DATE_MILADI'] = pd.to_datetime(df['DATE_MILADI'])

# استخراج تاریخ و ساعت
df['DATE'] = df['DATE_MILADI'].dt.date
df['HOUR'] = df['DATE_MILADI'].dt.hour

# فیلتر داده‌ها از ماه آخر 2023 (دسامبر) به بعد
df = df[(df['DATE_MILADI'].dt.year > 2023) | 
        ((df['DATE_MILADI'].dt.year == 2023) & (df['DATE_MILADI'].dt.month == 12))].copy()

# مرتب‌سازی داده‌ها بر اساس تاریخ و ساعت
df = df.sort_values(['DATE', 'HOUR']).reset_index(drop=True)

# ایجاد ستون MA و مقداردهی اولیه با NaN
df['MA'] = None

# لیست تاریخ‌های یکتا
unique_dates = sorted(df['DATE'].unique())

# دیکشنری برای نگه داشتن آخرین مقدار معتبر هر ساعت
last_valid_ma = {hour: None for hour in range(24)}

# محاسبه میانه 14 روز گذشته برای هر روز (از روز پانزدهم به بعد)
for i in range(14, len(unique_dates)):
    current_date = unique_dates[i]
    prev_dates = unique_dates[i-14:i]  # 14 روز قبلی
    
    for hour in range(24):
        # فیلتر رکوردهای 14 روز قبلی برای همان ساعت
        mask = (df['DATE'].isin(prev_dates)) & (df['HOUR'] == hour)
        values = df.loc[mask, 'POWER'].dropna().tolist()
        
        # حذف مقادیر صفر
        values = [v for v in values if v != 0]
        
        if len(values) > 0:
            # حذف 20 درصد پایین‌ترین مقادیر
            values_sorted = sorted(values)
            cutoff = int(len(values_sorted) * 0.2)
            filtered_values = values_sorted[cutoff:] if cutoff > 0 else values_sorted
            
            # محاسبه میانه
            median_power = np.median(filtered_values) if len(filtered_values) > 0 else None
        else:
            median_power = None
        
        # اگر مقدار محاسبه‌شده None یا صفر بود، آخرین مقدار معتبر قبلی جایگزین شود
        if median_power is None or median_power == 0:
            median_power = last_valid_ma[hour]
        
        # ذخیره مقدار در ستون MA
        df.loc[(df['DATE'] == current_date) & (df['HOUR'] == hour), 'MA'] = median_power
        
        # به‌روزرسانی آخرین مقدار معتبر برای این ساعت
        if median_power is not None and median_power != 0:
            last_valid_ma[hour] = median_power

# در انتها: اگر ستون EBRAZ صفر باشد، ستون MA هم صفر شود
df.loc[df['ebraz'] == 0, 'MA'] = 0

# تاریخ‌ها را یک روز جلو ببریم (پیش‌بینی فردا)
df['DATE_MILADI'] = df['DATE_MILADI'] + pd.Timedelta(days=1)
df['DATE'] = df['DATE_MILADI'].dt.date

# ذخیره فایل جدید (نام بدون تغییر)
df.to_csv("merged_output_with_MA_2024_median_nozeros.csv", index=False)

print("✅ Done. Saved MA predictions shifted to next day in merged_output_with_MA_2024_median_nozeros.csv")


✅ Done. Saved MA predictions shifted to next day in merged_output_with_MA_2024_median_nozeros.csv
